In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
scale = 2
scale_size = 1

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, scale_size):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, int(120/scale_size))
        self.fc2 = nn.Linear(int(120/scale_size), int(84/scale_size))
        self.fc3 = nn.Linear(int(84/scale_size), 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net(scale_size)

In [6]:
mask = list()
for i, param in enumerate(net.parameters()):
    if i<4:
        m = np.ones((param.detach().numpy()).shape)
    else:
        m = np.random.binomial(1, 1/scale, size=(param.detach().numpy()).shape)
    mask.append(torch.tensor(m))

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
losses = list()


for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        for p, m in zip(net.parameters(), mask):
            p.grad *= m
        
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            losses.append(running_loss / 2000)
            running_loss = 0.0


            
print('Finished Training')

np.save('loss_hist/losses_'+str(int(scale))+'_'+str(scale_size)+'.npy', np.array(losses))

[1,  2000] loss: 2.292
[1,  4000] loss: 2.038
[1,  6000] loss: 1.790
[1,  8000] loss: 1.661
[1, 10000] loss: 1.599
[1, 12000] loss: 1.529
[2,  2000] loss: 1.462
[2,  4000] loss: 1.427
[2,  6000] loss: 1.378
[2,  8000] loss: 1.348
[2, 10000] loss: 1.338
[2, 12000] loss: 1.289
[3,  2000] loss: 1.239
[3,  4000] loss: 1.247
[3,  6000] loss: 1.233
[3,  8000] loss: 1.207
[3, 10000] loss: 1.222
[3, 12000] loss: 1.218
[4,  2000] loss: 1.139
[4,  4000] loss: 1.151
[4,  6000] loss: 1.142
[4,  8000] loss: 1.124
[4, 10000] loss: 1.155
[4, 12000] loss: 1.117
[5,  2000] loss: 1.052
[5,  4000] loss: 1.069
[5,  6000] loss: 1.065
[5,  8000] loss: 1.072
[5, 10000] loss: 1.083
[5, 12000] loss: 1.067
[6,  2000] loss: 0.987
[6,  4000] loss: 1.022
[6,  6000] loss: 1.037
[6,  8000] loss: 1.009
[6, 10000] loss: 1.026
[6, 12000] loss: 1.015
[7,  2000] loss: 0.943
[7,  4000] loss: 0.967
[7,  6000] loss: 0.983
[7,  8000] loss: 0.982
[7, 10000] loss: 1.007
[7, 12000] loss: 0.968
[8,  2000] loss: 0.903
[8,  4000] 

In [9]:
outputs = net(inputs)
loss = criterion(outputs, labels)
loss.backward()

In [10]:
print(inputs.grad)

None


In [11]:
grads = []
for param in net.parameters():
    grads.append(param.grad.view(-1))

In [12]:
paras = []
for param in net.parameters():
    paras.append(param.view(-1))

In [13]:
PATH = './cifar_net_'+str(int(scale))+'_'+str(scale_size)+'.pth'
torch.save(net.state_dict(), PATH)

In [14]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# # print images
# imshow(torchvision.utils.make_grid(images))
# print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [16]:
net = Net(scale_size)
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [17]:
outputs = net(images)

In [18]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Predicted:    cat  ship  ship plane


In [19]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 62 %


In [20]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 72 %
Accuracy of   car : 74 %
Accuracy of  bird : 44 %
Accuracy of   cat : 39 %
Accuracy of  deer : 71 %
Accuracy of   dog : 35 %
Accuracy of  frog : 76 %
Accuracy of horse : 67 %
Accuracy of  ship : 71 %
Accuracy of truck : 72 %


In [21]:
from scipy import stats
import pickle

paras = list()
dists = list()
for param in net.parameters():
    paras.append((param.view(-1)).detach().numpy())
    weights = np.squeeze(paras[-1].flatten())
    dists.append(stats.gaussian_kde(weights))
    
pickle.dump( dists, open( "dists.pkl", "wb" ) )

In [22]:
# for p, d in zip(paras, dists):
#     x_plot = np.linspace(p.min(), p.max(), 100, endpoint=True)
#     plt.figure()
#     plt.hist(p, bins=100, density=True)
#     plt.plot(x_plot, d.pdf(x_plot))
#     plt.show()